In [2]:
import json
import pandas as pd

with open('../nflscraping/json/espn_scores.json') as rosters:
    data = json.load(rosters)

df = pd.json_normalize(data)
df

,season,week,awayteam,hometeam,awayscore,homescore,gamecast,boxscore,idgame,awayteam global record,awayteam away record,hometeam global record,hometeam home record
0,2020,1,Texans,Chiefs,20,34,https://espn.com/nfl/game/_/gameId/401220225,https://espn.com/nfl/boxscore/_/gameId/401220225,401220225,0-1,0-1,1-0,1-0
1,2020,1,Seahawks,Falcons,38,25,https://espn.com/nfl/game/_/gameId/401220313,https://espn.com/nfl/boxscore/_/gameId/401220313,401220313,1-0,1-0,0-1,0-1
2,2020,1,Jets,Bills,17,27,https://espn.com/nfl/game/_/gameId/401220116,https://espn.com/nfl/boxscore/_/gameId/401220116,401220116,0-1,0-1,1-0,1-0
3,2020,1,Bears,Lions,27,23,https://espn.com/nfl/game/_/gameId/401220282,https://espn.com/nfl/boxscore/_/gameId/401220282,401220282,1-0,1-0,0-1,0-1
4,2020,1,Packers,Vikings,43,34,https://espn.com/nfl/game/_/gameId/401220300,https://espn.com/nfl/boxscore/_/gameId/401220300,401220300,1-0,1-0,0-1,0-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1289,2021,18,49ers,Rams,27,24,https://espn.com/nfl/game/_/gameId/401326599,https://espn.com/nfl/boxscore/_/gameId/401326599,401326599,10-7,6-3,12-5,5-3
1290,2021,18,Patriots,Dolphins,24,33,https://espn.com/nfl/game/_/gameId/401326592,https://espn.com/nfl/boxscore/_/gameId/401326592,401326592,10-7,6-2,9-8,6-3
1291,2021,18,Seahawks,Cardinals,38,30,https://espn.com/nfl/game/_/gameId/401326597,https://espn.com/nfl/boxscore/_/gameId/401326597,401326597,7-10,4-5,11-6,3-5
1292,2021,18,Panthers,Buccaneers,17,41,https://espn.com/nfl/game/_/gameId/401326596,https://espn.com/nfl/boxscore/_/gameId/401326596,401326596,5-12,3-6,13-4,7-1


In [42]:
#def streak(idteam,idgame):
#  if
#df.loc[df['awayteam']=='Texans' or df['hometeam']=='Texans']

#df['homestreak'] = 
df['week'] = df['week'].astype(int)
df['awayscore'] = df['awayscore'].astype(float)
df['homescore'] = df['homescore'].astype(float)
df = df.loc[((df.awayteam=='Texans') | (df.hometeam=='Texans')) & (df.season=='2020')]
df.sort_values(by = 'week')
df.reset_index()

#df.info()



for index, row in df.iterrows():
  if(df['awayscore']<<df['homescore']):
    df['homestreak'] = -1
    df['awaystreak'] = 1

#for index,row in df.iterrows():
#for i in df:
  #if df[]
  #if df.at[index,'week']==1:


    #if(df['awayscore']<<df['homescore']):
    #  df['homestreak'] = -1
    #  df['awaystreak'] = 1
  #elif df.at[index,'week']==2:
    #df.loc[(df.)]
    #df.at[index,'streak'] = '83'
    #df.at[index,'streak'] = d
  #else:
  #  df['homestreak'] = 0
  #  df['awaystreak'] = 0

#for wk in df:



TypeError: unsupported operand type(s) for <<: 'Series' and 'Series'